In [2]:
import pandas as pd
import numpy as np
import csv
import json
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import scipy.stats as sp

# Download

In [3]:
def get_code(group, region, demographic, iterand):
    REGION_MAP = {"north": ['A', 'B'], "south": ['C', 'D']}
    CATEGORY_MAP = {"musician": "X", "non-musician": "Y"}
    filename = str(group)+REGION_MAP[region][iterand] + CATEGORY_MAP[demographic]+str(group)
    return filename

# Pairwise Data

In [16]:
# Opening JSON file
f = open('./json/pairwise.json')
  
# returns JSON object as 
data = json.load(f)

f.close()

for item in data:
    user = item['username']
    similarity = item['similarity']
    # open the file in the write mode
    with open('./output/india/pairwise/raw/'+str(user)+'.csv', 'w', newline='\n') as f:
        rows = similarity.split('\n')
        for row in rows:
            characters = row.split(',')
            line = []
            for x in characters:
                try:
                    line.append(int(x))
                except:
                    line.append('')
            writer = csv.writer(f)
            writer.writerow(line)

In [180]:
def generate_id(demographic, group):
    if demographic == 'north':
        ids = ['AX', 'BX', 'AY', 'BY']
    elif demographic == 'south':
        ids = ['CX', 'DX', 'CY', 'DY']
    elif demographic == 'musician':
        ids = ['AX', 'BX', 'CX', 'DX']
    elif demographic == 'non':
        ids = ['AY', 'BY', 'CY', 'DY']
    else:
        ids = ['AX', 'BX', 'CX', 'DX', 'AY', 'BY', 'CY', 'DY']
    return [str(group)+element+str(group) for element in ids]

In [191]:
def average_pair_group_data(first_id, participants):
    PATH = './output/india/pairwise/raw/'
    sum_df = pd.read_csv(PATH+first_id+'.csv', header=None).fillna(0)
    for pid in participants:
        df2 = pd.read_csv(PATH+pid+'.csv', header=None).fillna(0)
        sum_df = sum_df.add(df2, fill_value=0)
    average = sum_df.div(len(participants)+1)
    return average

In [200]:
def save_pair_data(category):
    empty_df = pd.DataFrame(index=range(30),columns=range(30)).fillna(0)

    group_0_ids = generate_id(category,0)
    group_0 = average_pair_group_data(group_0_ids[0], group_0_ids[1:])
    empty_df.iloc[0:5,0:5] = group_0

    group_1_ids = generate_id(category,1)
    group_1 = average_pair_group_data(group_1_ids[0], group_1_ids[1:])
    empty_df.iloc[5:10,5:10] = group_1

    group_2_ids = generate_id(category,2)
    group_2 = average_pair_group_data(group_2_ids[0], group_2_ids[1:])
    empty_df.iloc[10:15,10:15] = group_2


    group_3_ids = generate_id(category, 3)
    group_3 = average_pair_group_data(group_3_ids[0], group_3_ids[1:])
    empty_df.iloc[15:20,15:20] = group_3

    group_4_ids = generate_id(category, 4)
    group_4 = average_pair_group_data(group_4_ids[0], group_4_ids[1:])
    empty_df.iloc[20:25,20:25] = group_4

    group_5_ids = generate_id(category, 5)
    group_5 = average_pair_group_data(group_5_ids[0], group_5_ids[1:])
    empty_df.iloc[25:30,25:30] = group_5

    empty_df.to_csv('./output/india/pairwise/full_'+category+'.csv', index=None)

In [201]:
save_pair_data('all')
save_pair_data('north')
save_pair_data('south')
save_pair_data('musician')
save_pair_data('non')

# Feature Data

In [27]:
# Opening JSON file
f = open('./json/evaluation.json')
data = json.load(f)
f.close()

for participant in data:
    username = participant['username']
    features = participant['matrix']
    df = pd.DataFrame(features)
    df.to_csv('./output/india/feature/raw/'+username+'.csv', index=False)

# Bollywood Data

In [28]:
# Opening JSON file
f = open('./json/bollywood.json')
data = json.load(f)
f.close()

for participant in data:
    username = participant['username']
    features = participant['matrix']
    df = pd.DataFrame(features)
    df.to_csv('./output/india/bollywood/raw/'+username+'.csv', index=False)

# Formatting

In [46]:
def get_feature_data(code):
    PATH = './output/india/feature/raw/'
    df = pd.read_csv(PATH+code+'.csv')
    return df

def average_group_data(first_id, participants):
    sum_df = get_feature_data(first_id).drop(["song_id", "edited"], axis=1)
    for pid in participants:
        df2 = get_feature_data(pid).drop(["song_id", "edited"], axis=1)
        sum_df = sum_df.add(df2, fill_value=0)
    average = sum_df.div(len(participants)+1)
    return average

In [47]:
def generate_id(demographic, group):
    if demographic == 'north':
        ids = ['AX', 'BX', 'AY', 'BY']
    elif demographic == 'south':
        ids = ['CX', 'DX', 'CY', 'DY']
    elif demographic == 'musician':
        ids = ['AX', 'BX', 'CX', 'DX']
    elif demographic == 'non':
        ids = ['AY', 'BY', 'CY', 'DY']
    else:
        ids = ['AX', 'BX', 'CX', 'DX', 'AY', 'BY', 'CY', 'DY']
    return [str(group)+element+str(group) for element in ids]

# All Evaluation

In [103]:
def generate_full_feature(category):
    group_0_ids = generate_id(category, 0)
    group_0 = average_group_data(group_0_ids[0], group_0_ids[1:])

    group_1_ids = generate_id(category, 1)
    group_1 = average_group_data(group_1_ids[0], group_1_ids[1:])

    group_2_ids = generate_id(category, 2)
    group_2 = average_group_data(group_2_ids[0], group_2_ids[1:])

    group_3_ids = generate_id(category, 3)
    group_3 = average_group_data(group_3_ids[0], group_3_ids[1:])

    group_4_ids = generate_id(category, 4)
    group_4 = average_group_data(group_4_ids[0], group_4_ids[1:])

    group_5_ids = generate_id(category, 5)
    group_5 = average_group_data(group_5_ids[0], group_5_ids[1:])

    df = pd.concat([group_0, group_1, group_2, group_3, group_4, group_5]).reset_index(drop=True)
    df = df.round(2)

    df.to_csv('./output/india/feature/full_'+category+'.csv', index=None)

In [104]:
generate_full_feature('all')
generate_full_feature('north')
generate_full_feature('south')
generate_full_feature('musician')
generate_full_feature('non')

## Individual Distance Matrices

In [81]:
def get_feature_distance(input_data):
    data = squareform(pdist(input_data, metric='euclidean'))
    scaler = MinMaxScaler()
    scaler.fit(data)
    return scaler.transform(data).round(4)

In [222]:
feature_names = ['Ornamentation', 'Grooviness', 'Familiarity', 'Liking', 'Consonance', 'Valence', 'Excitement', 'Vocal Range', 'Sound Quality', 'Tempo', 'Rhythmic Regularity', 'Vocal Tension', 'Vocal Texture']

feature_mappings = {
    'overlap':'vocal_texture',
    'creativity':'familiarity',
    'likeness':'liking',
    'tempo':'tempo',
    'consonance':'consonance',
    'emotion':'valence',
    'decoration':'ornamentation',
    'range':'vocal_range',
    'quality':'sound_quality',
    'rhythm':'rhythmic_regularity',
    'excitingness':'excitement',
    'groove':'grooviness',
    'timbre':'vocal_tension'
}

full_feature = pd.read_csv('./output/india/feature/full_all.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/feature-distance/'+feature_mappings[feature]+'_full.csv')
    
north = pd.read_csv('./output/india/feature/full_north.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/feature-distance/'+feature_mappings[feature]+'_north.csv')

south = pd.read_csv('./output/india/feature/full_south.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/feature-distance/'+feature_mappings[feature]+'_south.csv')

musician = pd.read_csv('./output/india/feature/full_musician.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/feature-distance/'+feature_mappings[feature]+'_musician.csv')

non = pd.read_csv('./output/india/feature/full_non_musician.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/feature-distance/'+feature_mappings[feature]+'_non_musician.csv')

# Bollywood

In [223]:
def generate_bollywood_id(demographic, group):
    if demographic == 'north':
        ids = ['AX', 'BX', 'AY', 'BY', 'EX', 'EY', 'FX', 'FY']
    elif demographic == 'south':
        ids = ['CX', 'DX', 'CY', 'DY', 'GX', 'HX', 'GY', 'HY']
    elif demographic == 'musician':
        ids = ['AX', 'BX', 'CX', 'DX', 'EX', 'FX', 'GX', 'HX']
    elif demographic == 'non':
        ids = ['AY', 'BY', 'CY', 'DY', 'EY', 'FY', 'GY', 'HY']
    else:
        ids = ['AX', 'BX', 'CX', 'DX', 'EX', 'FX', 'GX', 'HX', 'AY', 'BY', 'CY', 'DY', 'EY', 'FY', 'GY', 'HY']
    return [str(group)+element+str(group) for element in ids]

In [224]:
def get_bollywood_feature_data(code):
    PATH = './output/india/bollywood/raw/'
    df = pd.read_csv(PATH+code+'.csv')
    return df

In [225]:
def average_bollywood_group_data(first_id, participants):
    sum_df = get_bollywood_feature_data(first_id).drop(["id", "edited"], axis=1)
    total = len(participants)+1
    for pid in participants:
        try:
            df2 = get_bollywood_feature_data(pid).drop(["id", "edited"], axis=1)
            sum_df = sum_df.add(df2, fill_value=0)
        except:
            total = total - 1
    average = sum_df.div(total)
    return average

In [226]:
def generate_full_bollywood_feature(category):
    group_0_ids = generate_bollywood_id(category, 0)
    group_0 = average_bollywood_group_data(group_0_ids[0], group_0_ids[1:])

    group_1_ids = generate_bollywood_id(category, 1)
    group_1 = average_bollywood_group_data(group_1_ids[0], group_1_ids[1:])

    group_2_ids = generate_bollywood_id(category, 2)
    group_2 = average_bollywood_group_data(group_2_ids[0], group_2_ids[1:])

    group_3_ids = generate_bollywood_id(category, 3)
    group_3 = average_bollywood_group_data(group_3_ids[0], group_3_ids[1:])

    df = pd.concat([group_0, group_1, group_2, group_3]).reset_index(drop=True)
    df = df.round(2)
    df.to_csv('./output/india/bollywood/full_'+category+'.csv', index=None)

In [135]:
generate_full_bollywood_feature('all')
generate_full_bollywood_feature('north')
generate_full_bollywood_feature('south')
generate_full_bollywood_feature('musician')
generate_full_bollywood_feature('non')

In [137]:
full_feature = pd.read_csv('./output/india/bollywood/full_all.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/bollywood-feature-distance/'+feature+'_full.csv')
    
north = pd.read_csv('./output/india/feature/full_north.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/bollywood-feature-distance/'+feature+'_north.csv')

south = pd.read_csv('./output/india/feature/full_south.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/bollywood-feature-distance/'+feature+'_south.csv')

musician = pd.read_csv('./output/india/feature/full_musician.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/bollywood-feature-distance/'+feature+'_musician.csv')

non = pd.read_csv('./output/india/feature/full_non_musician.csv')
for feature in full_feature.columns:
    feature_df = pd.DataFrame(get_feature_distance(full_feature[[feature]]))
    feature_df.to_csv('./output/india/bollywood-feature-distance/'+feature+'_non_musician.csv')

In [ ]:
full_feature = pd.read_csv('./output/india/bollywood/full_all.csv')

# Flattened Distance Matrix

In [206]:
def flatten_distance_matrix(numpy_array):
    arr = []
    for i in range(5):
        for j in range(5):
            if i<j:
                arr.append(numpy_array[i,j])
    return arr

In [207]:
PATH = './output/japan/similarity/'

full_partial = []
for i in range(6):
    npar = pd.read_csv(PATH+str(i)+'_average_distance.csv').to_numpy()
    x = flatten_distance_matrix(npar)
    full_partial.extend(x)
full_partial = [x/100 for x in full_partial]

In [235]:
PATH = './output/japan/feature/'
feature_names = ['Ornamentation', 'Grooviness', 'Familiarity', 'Liking', 'Consonance', 'Valence', 'Excitement', 'Vocal Range', 'Sound Quality', 'Tempo', 'Rhythmic Regularity', 'Vocal Tension', 'Vocal Texture']
output = pd.DataFrame()

for i in range(0, len(feature_names)+1):
    label = feature_names[i-1].lower().replace(' ', '_')
    npar = pd.read_csv(PATH+label+'_average.csv').to_numpy()
    flattened_feature = []
    for j in range(6):
        flattened = flatten_distance_matrix(npar[j*5:(j*5)+5, j*5:(j*5)+5])
        flattened_feature.extend(flattened)
    r, p = stats.pearsonr(flattened_feature, full_partial)
    output = output.append({'Feature':feature_names[i-1], 'r':r, 'p-value':p}, ignore_index=True)
output = output.round(2)
output.to_csv('./output/analysis/japan_feature_pairwise.csv', index=None)

/var/folders/t5/g46h2p3n4bx62qf44whk03240000gp/T/ipykernel_97152/2467910764.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'Feature':feature_names[i-1], 'r':r, 'p-value':p}, ignore_index=True)
/var/folders/t5/g46h2p3n4bx62qf44whk03240000gp/T/ipykernel_97152/2467910764.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'Feature':feature_names[i-1], 'r':r, 'p-value':p}, ignore_index=True)
/var/folders/t5/g46h2p3n4bx62qf44whk03240000gp/T/ipykernel_97152/2467910764.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'Feature':feature_names[i-1], 'r':r, 'p-value':p}, ignore_index=True)
/var/folders/t5/g46h2p3n4bx62qf44whk03240000gp/T/ipykernel_97152

In [236]:
PATH = './output/india/pairwise/'

indian_partial = []
for i in range(6):
    npar = pd.read_csv(PATH+'full_all.csv').to_numpy()
    x = flatten_distance_matrix(npar[i*5:(i*5)+5, i*5:(i*5)+5])
    indian_partial.extend(x)

indian_partial = [1- (x / 100) for x in indian_partial]

output = pd.DataFrame()
PATH = './output/india/feature-distance/'
feature_names = ['Ornamentation', 'Grooviness', 'Familiarity', 'Liking', 'Consonance', 'Valence', 'Excitement', 'Vocal Range', 'Sound Quality', 'Tempo', 'Rhythmic Regularity', 'Vocal Tension', 'Vocal Texture']
labels = [x.lower().replace(' ', '_') for x in feature_names]
for i in range(0, len(labels)):
    label = labels[i]
    npar = pd.read_csv(PATH+label+'_full.csv').to_numpy()
    flattened_feature = []
    for j in range(6):
        flattened = flatten_distance_matrix(npar[j*5:(j*5)+5, j*5:(j*5)+5])
        flattened_feature.extend(flattened)
    r, p = stats.pearsonr(flattened_feature, indian_partial)
    output = output.append({'Feature':feature_names[i-1], 'r':r, 'p-value':p}, ignore_index=True)
output = output.round(2)
output.to_csv('./output/analysis/india_feature_pairwise.csv', index=None)

/var/folders/t5/g46h2p3n4bx62qf44whk03240000gp/T/ipykernel_97152/2178472131.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'Feature':feature_names[i-1], 'r':r, 'p-value':p}, ignore_index=True)
/var/folders/t5/g46h2p3n4bx62qf44whk03240000gp/T/ipykernel_97152/2178472131.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'Feature':feature_names[i-1], 'r':r, 'p-value':p}, ignore_index=True)
/var/folders/t5/g46h2p3n4bx62qf44whk03240000gp/T/ipykernel_97152/2178472131.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'Feature':feature_names[i-1], 'r':r, 'p-value':p}, ignore_index=True)
/var/folders/t5/g46h2p3n4bx62qf44whk03240000gp/T/ipykernel_97152

In [66]:
def get_japanese_feature_data(code):
    PATH = './data/japanese/evaluation/evaluation_'
    df = pd.read_csv(PATH+code+'.csv', header=None).drop([0,14], axis=1)
    return df

columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

participants = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

full_japan = pd.DataFrame(columns = columns)
for i in range(6):
    sum_df = pd.DataFrame(columns = columns)
    for pid in participants:
        df = get_japanese_feature_data(str(i)+pid)
        sum_df = sum_df.add(df, fill_value=0)
    average = sum_df.div(len(participants))
    full_japan = pd.concat([full_japan,average]).reset_index(drop=True)

In [69]:
full_japan.to_csv('./output/japan/feature/full_feature.csv')